In [1]:
#!/usr/bin/env python
# coding: utf-8

# In[5]:


# Data Transformation 
import numpy as np
import pandas as pd
def Transform(train_slices, test_slices):
    def process_transform_dataset(df):
        samples = []
        m = 100
        k = 6
        n = len(df.index)
        for i in range(n):
            x = df.iloc[i].tolist()
            sample = []
            for j in range(m):
                index = j 
                for p in range(k):
                    sample.append(x[index])
                    index = index + m
            sample.append(x[len(x)-1])
            samples.append(sample)
        return samples

    train_df = pd.DataFrame(train_slices)
    test_df = pd.DataFrame(test_slices)


    train_transform = process_transform_dataset(train_df)
    test_transform = process_transform_dataset(test_df)
    return train_transform, test_transform


# In[ ]:

In [10]:
import os
import numpy as np
import pandas as pd
import time
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn import metrics
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt

# Set random seeds for reproducibility
np.random.seed(1)
import random as rn
rn.seed(1)
import tensorflow as tf
tf.random.set_seed(1)

# Load data
train = pd.read_csv(os.path.join(os.getcwd(),'Input', 'Basic_Motion_Train.csv'))
test = pd.read_csv(os.path.join(os.getcwd(),'Input', 'Basic_Motion_Test.csv'))

# Trasformation
train_transform, test_transform = Transform(train, test)
train = pd.DataFrame(train_transform)
test = pd.DataFrame(test_transform)

# Preprocess the data
x_train, y_train = train.iloc[:, :-1].values, train.iloc[:, -1].values
x_test, y_test = test.iloc[:, :-1].values, test.iloc[:, -1].values

# Feature scaling
sc = StandardScaler()
x_train = sc.fit_transform(x_train).reshape(x_train.shape[0], 100, 6)
x_test = sc.transform(x_test).reshape(x_test.shape[0], 100, 6)

# One-hot encode labels
encoder = OneHotEncoder(sparse_output=False)
y_train = encoder.fit_transform(y_train.reshape(-1, 1))
y_test = encoder.transform(y_test.reshape(-1, 1))

# Set up directories and model parameters
base_directory = os.path.join(os.getcwd(), 'Output', 'LSTM')
os.makedirs(base_directory, exist_ok=True)

# Define model parameters and train
m, n, epochs, patience = 6, 6, 300, 4  # example values
subdirectory = os.path.join(base_directory, f"{m}.{n}.{epochs}.{patience}")
os.makedirs(subdirectory, exist_ok=True)

# Build the model
model = Sequential([
    LSTM(m, return_sequences=True, input_shape=(100, 6)),
    LSTM(n),
    Dense(4, activation='softmax')
])
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Callbacks
es = EarlyStopping(monitor='val_loss', mode='min', patience=patience)
cp = ModelCheckpoint(os.path.join(subdirectory, 'bestweights.keras'), monitor='val_loss', mode='min', save_best_only=True)

# Train the model
model.fit(x_train, y_train, validation_split=0.2, epochs=epochs, batch_size=12, callbacks=[es, cp])

# Evaluate the model
scores = model.evaluate(x_test, y_test, verbose=0)
print(f"Test Accuracy: {scores[1] * 100:.2f}%")

# Predict and calculate testing time
start_time = time.time()
y_pred = model.predict(x_test)
end_time = time.time()
testing_time = end_time - start_time
print(f"Testing Time: {testing_time} seconds")

# Save testing time
with open(os.path.join(subdirectory, 'testing_time.txt'), "w") as f:
    f.write(f"Testing Time: {testing_time} seconds")

# Save class probabilities
np.savetxt(os.path.join(subdirectory, 'class_probabilities.csv'), y_pred, delimiter=',')

# Classification report and confusion matrix
y_pred_labels = np.argmax(y_pred, axis=1)
y_test_labels = np.argmax(y_test, axis=1)

# Save classification report
report_str = classification_report(y_test_labels, y_pred_labels)
with open(os.path.join(subdirectory, 'classification_report.txt'), "w") as f:
    f.write(report_str)

# Confusion matrix plot
disp = metrics.ConfusionMatrixDisplay.from_predictions(y_test_labels, y_pred_labels)
disp.figure_.suptitle("Confusion Matrix")
disp.figure_.savefig(os.path.join(subdirectory, 'Confusion_Matrix.png'))
plt.close()


Epoch 1/300


C:\Users\tenzi\anaconda3\envs\aeon-env\lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


4/4 ━━━━━━━━━━━━━━━━━━━━ 5s 324ms/step - accuracy: 0.3712 - loss: 1.3610 - val_accuracy: 0.4545 - val_loss: 1.3246
Epoch 2/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step - accuracy: 0.3942 - loss: 1.3522 - val_accuracy: 0.5455 - val_loss: 1.3180
Epoch 3/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step - accuracy: 0.3942 - loss: 1.3453 - val_accuracy: 0.5455 - val_loss: 1.3109
Epoch 4/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step - accuracy: 0.3942 - loss: 1.3385 - val_accuracy: 0.5455 - val_loss: 1.3032
Epoch 5/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step - accuracy: 0.3942 - loss: 1.3317 - val_accuracy: 0.5455 - val_loss: 1.2951
Epoch 6/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step - accuracy: 0.3942 - loss: 1.3246 - val_accuracy: 0.5455 - val_loss: 1.2863
Epoch 7/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step - accuracy: 0.3942 - loss: 1.3171 - val_accuracy: 0.5455 - val_loss: 1.2769
Epoch 8/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step - accuracy: 0.3942 - loss: 1.3092 - val_accuracy: 0.5455 - val_loss: 1.2669
E

array([[[-2.32983461e-01, -3.92732326e-01, -5.32112083e-01,
          2.06051902e+00,  8.54102303e-01,  9.67860719e-01],
        [-2.41950059e-01, -3.26810347e-01, -5.36888171e-01,
          2.04454534e+00,  8.55461890e-01,  9.66423140e-01],
        [-3.14479303e-01, -5.72235913e-02,  6.29138993e-01,
          2.22396591e-01,  1.62751221e-01, -1.47144443e-01],
        ...,
        [-1.81509189e-01,  7.37129981e-01,  9.20976701e-03,
          1.34971821e-01,  3.36731645e-03,  4.80813544e-01],
        [-5.52556003e-02,  9.65441341e-01,  9.66674037e-02,
          1.30201387e-01, -5.96471223e-02,  4.86479073e-01],
        [-1.34115667e-01,  9.48828707e-01, -6.51093194e-03,
         -2.59750315e-01,  2.43683539e-01, -5.34127748e-03]],

       [[ 6.48717076e-01,  3.00300732e+00, -3.93252645e-01,
         -1.29342862e+00,  3.41317117e-01,  5.59568999e-02],
        [ 6.46710745e-01,  2.84678773e+00, -3.98942332e-01,
         -1.26232645e+00,  3.43318819e-01,  6.40100546e-02],
        [-8.35167